In [4]:
!pip install -q datasets torch pytorch-lightning transformers

### 0. Сбор данных

То, как это собирается сейчас, очень неудобно, я перепишу по-человечески. Мне сами тексты не нужны, но мне нужны последовательности

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from common_utils import correct_aspects, asp_mapper, sent_mapper

Сначала это надо всё считать. Я сделаю точно такой же сплит, но зафиксирую сид, размер теста = 0.25. Проблема в том, что у меня тексты из одного распределения, это не поможет

Ещё беда в том, что токены не имеют индекса, я не могу нормально там всё заджойнить. Токены в аспектах и в текстах после эксплоуда не совпадают из-за пунктуации и прочего мусора. Я пофиксил это как смог в `correct_aspects` - если можно разбить целый аспект на несколько маленьких, я так и делаю. Иначе я не смогу их смёрджить и у меня поедет разметка

In [3]:
aspects = correct_aspects(pd.read_csv(
    "train_aspects.txt", delimiter="\t",
    names=["text_id", "aspect", "token", "begin", "end", "sentiment"]
))
aspects["aspect"] = aspects.aspect.map(asp_mapper)
aspects["sentiment"]= aspects.sentiment.map(sent_mapper)

texts = pd.read_csv(
    "train_reviews.txt", delimiter="\t",
    names=["text_id", "token"]
)
ids = texts.text_id.unique()
train_ids, valid_ids = train_test_split(ids, random_state=69, shuffle=True)

labels = pd.read_csv(
     "train_cats.txt", delimiter="\t",
    names=["total_asp", "total_sent"]
).reset_index(names=["text_id"])
labels["total_sent"] = labels.total_sent.map(sent_mapper)
labels = labels.groupby("text_id").agg(lambda x: [y for y in x]).reset_index()

texts["text"] = texts["token"]
texts = texts.sort_values("text_id")
texts["token"] = texts["token"].map(lambda x: x.split())
texts = texts.explode("token").reset_index(drop=True)

In [8]:
# последние 2 строки - после разбиения одного аспекта, видно по begin-end
aspects.head()

,text_id,aspect,token,begin,end,sentiment
0,3976,0,ресторане,71,80,3
1,3976,0,ресторанах,198,208,3
2,3976,0,ресторане,256,265,3
3,3976,2,Столик,267,273,3
4,3976,2,бронировали,274,285,3


Теперь надо сам индекс добавить и в тексты тоже. Этим вообще должны заниматься те, кто бд делает, но ладно, мне не сложно

In [4]:
from string import punctuation

text_id = 0
for idx, row in texts.iterrows():
    
    if row["text_id"] != text_id:
        text_id = row["text_id"]
        char_id = 0
    texts.loc[idx, "begin"] = char_id
    token = row["token"]
    texts.loc[idx, "end"] = char_id + len(token.strip(punctuation))
    
    char_id += len(token) + 1
    
texts[["begin", "end"]] = texts[["begin", "end"]].astype(int)
texts["token"] = texts.token.apply(lambda x: x.strip(punctuation))

Сами тексты мне тут не нужны, но по памяти это вообще не бьёт - слишком маленькие размеры, я забил. Мёрджим и делаем датасет

In [10]:
df = pd.merge(
    texts.drop("text", axis=1),
    aspects,
    on=["text_id", "token", "begin", "end"],
    how="left"
) \
# .rename(columns={"begin_y": "begin", "end_y": "end"}) \
# .drop_duplicates(["begin", "end"]) \
# .sort_values(["text_id", "begin", "end"])
        
df["aspect"].fillna(5, inplace=True)
df["sentiment"].fillna(5, inplace=True)

valid = df[df.text_id.isin(valid_ids)]
train = df[df.text_id.isin(train_ids)]

train_dataset = train.groupby("text_id").agg(lambda x: [y for y in x]).reset_index()
valid_dataset = valid.groupby("text_id").agg(lambda x: [y for y in x]).reset_index()
train_dataset = pd.merge(train_dataset, labels[["text_id", "total_sent"]], on="text_id", how="left")
valid_dataset = pd.merge(valid_dataset, labels[["text_id", "total_sent"]], on="text_id", how="left")

valid_dataset.explode(["token", "begin", "end", "aspect", "sentiment"]).to_csv("valid_meta.csv", index=0)

train_dataset = Dataset.from_pandas(train_dataset)
valid_dataset = Dataset.from_pandas(valid_dataset)

Теперь надо эту красоту токенизировать и сопоставить все аспекты. Было бы удобнее сразу конечно это сделать, а потом бить, чтобы избежать копипасты, но это ради чистоты эксперимента, потому что задачку я чуть-чуть переделал

In [11]:
from bert import tokenize_and_align_labels
from torch.utils.data import DataLoader

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
train_dataset = train_dataset.remove_columns(["token", "begin", "end", "text_id", '__index_level_0__'])
train_dataset.set_format("pt")
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

valid_dataset = valid_dataset.map(tokenize_and_align_labels, batched=True)
valid_dataset.set_format("pt")

meta_cols = ["begin", "end"]
bert_valid_meta = pd.DataFrame(columns=meta_cols)
all_tokens = [y for x in valid_dataset["token"] for y in x if y != "-100"]
bert_valid_meta["token"] = all_tokens
for col in meta_cols:
    bert_valid_meta[col] = valid_dataset[col][valid_dataset[col] != -100].detach().numpy()
    
cur_idx = -1
for idx, row in bert_valid_meta.iterrows():
    if row["begin"] == 0:
        cur_idx += 1
    bert_valid_meta.loc[idx, "text_id"] = valid.text_id.unique()[cur_idx]
    
bert_valid_meta["text_id"] = bert_valid_meta["text_id"].astype(int)
bert_valid_meta = bert_valid_meta[["text_id", "token", "begin", "end"]]
bert_valid_meta.to_csv("bert_valid_meta.xsv", index=0)

valid_dataset = valid_dataset.remove_columns(["token", "begin", "end", "text_id", '__index_level_0__'])
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

Map:   0%|          | 0/213 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

### 1. Baseline

Но сначала надо вернуться на шаг назад и проделать всё, что было в бейзлайне ещё разок, чтоб было с чем сравнивать. Заодно проверим, что моя задачка не совпадает с тем, что делается в бейзлайне, но это и так понятно

In [13]:
%%capture
from baseline import normalize, get_mention_category, label_texts

valid_aspects = aspects[aspects.text_id.isin(valid_ids)]
train_aspects = aspects[~aspects.text_id.isin(valid_ids)]

valid_texts = texts[texts.text_id.isin(valid_ids)]
train_texts = texts[~texts.text_id.isin(valid_ids)]

train_aspects["norm_mention"] = train_aspects.token.apply(lambda x: tuple(normalize(x)))

best_mention_cat = get_mention_category(train_aspects, 'aspect')
best_mention_sentiment = get_mention_category(train_aspects, 'sentiment')

Дальше я это записываю точь в точь, как сделано в тетрадке с бейзлайном, а затем считываю обратно в формате датафрейма. Дальше останется посчитать аккураси

In [14]:
from tqdm.auto import tqdm

with open('valid_base_aspects.txt', 'w') as f:
    for text, idx in valid_texts[['text', 'text_id']].drop_duplicates().values:
        for asp in label_texts(text, best_mention_cat, best_mention_sentiment):
            print(idx, *asp, sep="\t", file=f)
            
# он есть в ноутбуке с бейзлайном, чтобы было максимально честно
# with open('valid_aspects.txt', 'w') as f:
#     for idx, l in valid_aspects.iterrows():
#         print(*l.values, sep="\t", file=f)
            
labels = correct_aspects(pd.read_csv(
    "valid_base_aspects.txt", delimiter="\t",
    names=["text_id", "aspect", "token", "begin", "end", "sentiment"]
))

У меня есть настоящие аспекты и точно такие же предсказанные аспекты. Можно их склеить элементарно, на всякий случай опять удостоверимся, что все токены совпадают через мёрдж

In [15]:
from functools import reduce

true_pred = reduce(
    lambda x, y: pd.merge(x, y, on=["text_id", "token", "begin", "end"], how="left"),
    [valid_texts, labels, valid_aspects]
).drop(["text"], axis=1)

# aspect5 и sentiment4 := None
true_pred["aspect_x"].fillna(5, inplace=True)
true_pred["aspect_y"].fillna(5, inplace=True)

true_pred["sentiment_x"].fillna(4, inplace=True)
true_pred["sentiment_y"].fillna(4, inplace=True)

true_pred[["aspect_x", "aspect_y"]] = true_pred[["aspect_x", "aspect_y"]].applymap(
    lambda x: {v: k for k, v in asp_mapper.items()}[x]
)
true_pred[["sentiment_x", "sentiment_y"]] = true_pred[["sentiment_x", "sentiment_y"]].applymap(
    lambda x: {v: k for k, v in sent_mapper.items()}[x]
)

С аккураси немножко сложно, я помню статью с прошлого года, поэтому у меня их целых 4 варианта. Разница в том, считаем мы наны или нет. Если оба нан, то проблем нет, они в аккураси не учитываются, а если один из? Есть 4 варианта - разрешаем левому, правому, обоим, ни одному. Но попробуем побить все 4

In [16]:
from inference import custom_accuracy, get_full_sentiment

answers = pd.DataFrame(columns=["label", "method", "model", "accuracy"])
for label in ["sentiment", "aspect"]:
    for how in ["pred", "true", "common", "excluding"]:
        answers.loc[answers.shape[0]] = {
            "method": how, "label": label, "model": "naïve",
            "accuracy": custom_accuracy(
                true_pred[f"{label}_x"], true_pred[f"{label}_y"], how=how
            ).item()
        }
        
with open('valid_base_packed_cats.txt', 'w') as f:
    for idx, c, s in get_full_sentiment(true_pred, "aspect_x", "sentiment_x"):
        print(idx, c, s, sep="\t", file=f)
        
answers

,label,method,model,accuracy
0,sentiment,pred,naïve,0.577543
1,sentiment,true,naïve,0.577543
2,sentiment,common,naïve,0.577543
3,sentiment,excluding,naïve,0.577543
4,aspect,pred,naïve,0.783345
5,aspect,true,naïve,0.244547
6,aspect,common,naïve,0.947099
7,aspect,excluding,naïve,0.232023


Если анализировать это дело, то выйдет вот что:
1. Аспекты. Если игнорировать наны полностью, то всё вообще прекрасно, почти всегда попадаем. Но судя по `excluding` очень часто выходимм за рамки, слишком много лишнего
2. Сентимент. Тут всё ещё хуже по обоим направлением, но в целом выводы все те же

Мета информацию про сами токены и позицию сохраним, она пригодится в дальнейшем

In [10]:
(
    true_pred[["text_id", "token", "begin", "end"]] \
    .sort_values("text_id") \
    .to_csv("valid_meta.csv", index=0)
)

И заодно можем посчитать, как это будет работать с исходным инференсом. Я сразу скажу, что я его не особо разбирал, там много строчек, может быть так действительно правильно, но меня интересовало сделать разметку на уровне токенов. Понятно, что для символов оно будет в целом так же, но тогда возникнут сложности с частью 2

Если не собирвать аспекты, а оставить их по токенам, то будет грустно. В принципе мы увидим то же самое, что я сказал выше, только не уверен, насколько там уместно называть это пресижном и реколлом

In [11]:
from inference import reference_check

reference_check(
    "valid_aspects.txt", "valid_base_aspects.txt",
    "valid_cats.txt", "valid_base_packed_cats.txt"
)

,None
Full match precision,0.17209398186314923
Full match recall,0.7142857142857143
Partial match ratio in pred,0.2384583676834295
Full category accuracy,0.0
Partial category accuracy,0.17209398186314923
Patial sentiment accuracy,0.0
Full sentiment accuracy,0.0
Overall sentiment accuracy,0.9464788732394366


А если собрать, то уже куда ни шло

In [12]:
from inference import pack

pack(
    true_pred, "aspect_x", "sentiment_x",
    'valid_base_packed_aspects.txt'
)

  0%|          | 0/4539 [00:00<?, ?it/s]

In [13]:
full_df = pd.concat([
    reference_check(
        "valid_aspects.txt", f"valid_{model}_aspects.txt",
        "valid_cats.txt", f"valid_{model}_cats.txt", model=model
    )
    for model in ["base_packed", "true_base"]
], axis=1)

full_df

,base_packed,true_base
Full match precision,0.1480654761904762,0.4146224146224146
Full match recall,0.34046193327630453,0.737382378100941
Partial match ratio in pred,0.33742559523809523,0.5108225108225108
Full category accuracy,0.1402529761904762,0.4078884078884079
Partial category accuracy,0.2734375,0.5036075036075036
Patial sentiment accuracy,0.6483300589390962,0.605
Full sentiment accuracy,0.6809045226130653,0.665893271461717
Overall sentiment accuracy,0.9464788732394366,0.4056338028169014


Получается, что уже достаточно лишь перейти к токенам, чтобы побить бейзлайн

Так или иначе, будем стараться побить всё, что здесь есть. Очевидно, наивный подход не самый лучшмй вармант, разве что у меня реально много данных, но это не наш случай. Хотя если смотреть на общий сентимент, то вообще-то всё очень даже неплохо, хотя если честно, это как раз самое сложное, не понимаю, почему считается самым простым, потому что у меня нет правильных ответов

### 2. BERT

Я взял вообще дефолтный берт и сразу решился дообучать его полностью, потому что могу это себе позволить. Все параметры стандартные, ничего хитрого
https://wandb.ai/lerostre/assez?workspace=user-lerostre

In [17]:
from bert import train, evaluate
import wandb
import torch
from pytorch_lightning.loggers import WandbLogger
from transformers import get_cosine_schedule_with_warmup
wandb.login()
torch.manual_seed(69)

wandb: Currently logged in as: lerostre. Use `wandb login --relogin` to force relogin


In [ ]:
wandb_logger = WandbLogger(
    project="assez", log_model='all', name="cat_only_bert"
)
optimizer_params = {
    "optimizer": torch.optim.Adam,
    "optimizer_kwargs": dict(lr=5e-5),
    "scheduler": get_cosine_schedule_with_warmup,
    "scheduler_kwargs": dict(num_warmup_steps=20, num_training_steps=200),
    "weights": [0, 0, 1]
}
training_arguments = {
    "max_epochs": 50,
    "accelerator": "gpu",
    "log_every_n_steps": 1,
    "logger": wandb_logger
}

train(
    optimizer_params, training_arguments,
    train_loader, valid_loader
)

In [7]:
wandb_logger = WandbLogger(project="assez", name="cat_bert")
# artifact = wandb_logger.use_artifact("lerostre/assez/model-1qlz2ava:best")
artifact = wandb_logger.use_artifact("lerostre/assez/model-z10m03iu:best")
artifact_dir = artifact.download()

wandb: Downloading large artifact model-z10m03iu:best, 2031.39MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.5


In [ ]:
bert_res = evaluate("artifacts/model-z10m03iu:v49/model.ckpt", valid_loader)

In [19]:
from bert import *

model = AspectBERT.load_from_checkpoint("artifacts/model-z10m03iu:v49/model.ckpt").to("cpu")
res = [model.step(batch, idx, log=False) for idx, batch in enumerate(valid_loader)]
aspects = ["aspect_true", "aspect_pred"]
sentiments = ["sentiment_true", "sentiment_pred"]
clfs = ["clf_true", "clf_pred"]
bert_res = pd.DataFrame(columns=aspects+sentiments)
clf_bert = pd.DataFrame(columns=clfs)

for batch in res:
    batch["clf_true"] = batch["clf_true"].reshape(-1, 5)
    batch["clf_pred"] = batch["clf_pred"].reshape(-1, 5)
    for row in tqdm(zip(*list(batch.values())[4:-2])):
        bert_res.loc[bert_res.shape[0]] = row
    for row in zip(*list(batch.values())[-2:]):
        clf_bert.loc[clf_bert.shape[0]] = row

bert_res[aspects+sentiments] = bert_res[aspects+sentiments].applymap(lambda x: x.item())
bert_res[aspects] = bert_res[aspects].applymap(
    lambda x: {v: k for k, v in asp_mapper.items()}[x]
)
bert_res[sentiments] = bert_res[sentiments].applymap(
    lambda x: {v: k for k, v in sent_mapper.items()}[x]
)
clf_bert[clfs] = clf_bert[clfs].applymap(
    lambda x: [y.item() for y in x]
)
clf_bert[clfs] = clf_bert[clfs].applymap(
    lambda x: list(map({v: k for k, v in sent_mapper.items()}.get, x))
)

/home/user/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [97]:
import numpy as np
from inference import custom_accuracy

for label in ["sentiment", "aspect"]:
    for how in ["pred", "true", "common", "excluding"]:
        answers.loc[answers.shape[0]] = {
            "method": how, "label": label, "model": "bert",
            "accuracy": custom_accuracy(
                bert_res[f"{label}_true"], bert_res[f"{label}_pred"],
                how=how).item()
        }
        
answers.loc[answers.groupby(["label", "method"])["accuracy"].idxmax()]

NameError: name 'answers' is not defined

Ну почти, в одной метрике не обогнал, но в общем-то и непонятно, почему мне нужно её брать. Я бы скорее смотрел на `common` или `excluding`

На самом деле результаты не сказать, чтобы впечатляющие. Понятно, что если дать сетке другие тексты, которых она при обучении не видела, будет скорее всего не очень, потому что я сильно переобучился под данные, ну это и так понятно. Строго говоря, других данных у меня нет, я использую только то, что дают и оно работет прекрасно. Теперь по метрикам
1. Аспекты выделяются почти идеально, если опустить наны. Если их не опускать, то видно, что с границами порядок примерно в 6 из 10 случаев, что тем не менее сильно лучше, чем раньше
2. Сентимент остался на самом деле на уровне, но с границами теперь всё в разы лучше, больше попаданий

Ну и глянем, как это выглядит, на исходной функции и с собранными н-граммами

In [20]:
from inference import pack, get_full_sentiment

valid_meta = pd.read_csv("bert_valid_meta.csv")[["text_id", "token", "begin", "end"]]
columns = ["text_id", "aspect_pred", "token", "begin", "end", "sentiment_pred"]

bert_pretty_res = pd.concat([valid_meta, bert_res], axis=1)[columns]
bert_pretty_res = bert_pretty_res.dropna().reset_index(drop=True)
bert_pretty_res[["text_id", "begin", "end"]] = bert_pretty_res[["text_id", "begin", "end"]].astype(int)

with open('valid_bert_aspects.txt', 'w') as f:
    for idx, l in bert_pretty_res.iterrows():
        print(*l.values, sep="\t", file=f)
        
with open('valid_bert_packed_cats.txt', 'w') as f:
    for idx, row in enumerate(clf_bert["clf_pred"].values):
        for cat, sent in zip(
            ['Food', 'Interior', 'Price', 'Whole', 'Service'], row
        ):
            print(valid_ids[idx], cat, sent, sep="\t", file=f)
        
pack(
    bert_pretty_res, "aspect_pred", "sentiment_pred",
    "valid_bert_packed_aspects.txt"
)

  0%|          | 0/1407 [00:00<?, ?it/s]

In [21]:
from inference import reference_check

full_df = pd.concat([
    reference_check(
        "dev_aspects.txt", f"valid_{model}_aspects.txt",
        "dev_cats.txt", f"valid_{model}_cats.txt", model=model
    )
    for model in ["base_packed", "true_base", "bert_packed"]
], axis=1)

full_df

,base_packed,true_base,bert_packed
Full match precision,0.1480654761904762,0.4146224146224146,0.6991071428571428
Full match recall,0.34046193327630453,0.737382378100941,0.669803250641574
Partial match ratio in pred,0.33742559523809523,0.5108225108225108,0.8232142857142857
Full category accuracy,0.1402529761904762,0.4078884078884079,0.6741071428571429
Partial category accuracy,0.2734375,0.5036075036075036,0.8160714285714286
Patial sentiment accuracy,0.6483300589390962,0.605,0.7338129496402878
Full sentiment accuracy,0.6809045226130653,0.665893271461717,0.756066411238825
Overall sentiment accuracy,0.12394366197183099,0.5492957746478874,0.6253521126760564


### 3. Inference

Инференс сделать достаточно легко, если датафреймы отзывов и аспектов 1 в 1, как те, что были даны на трейне, иначе что-то сломается. Я вообще не удивлюсь, если даже несмотря на это что-то сломается всё равно, уж очень данные специфические, неудобно с ними работать. Ниже код собственно воспроизводит всё то же, что и выше в ноутбуке, можно проверить

In [ ]:
from inference import inference

inference("valid_texts.csv")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

wandb: Currently logged in as: lerostre. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-z10m03iu:best, 2031.39MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.6
/home/user/conda/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [2]:
from inference import reference_check

reference_check(
    "dev_aspects.txt", "test_bert_packed_aspects.txt",
    "dev_cats.txt", "test_bert_cats.txt"
)

,None
Full match precision,0.07566638005159071
Full match recall,0.07527801539777587
Partial match ratio in pred,0.2226999140154772
Full category accuracy,0.018056749785038694
Partial category accuracy,0.10834049871023216
Patial sentiment accuracy,0.5263157894736842
Full sentiment accuracy,0.5340909090909091
Overall sentiment accuracy,0.1352112676056338
